# <h1 align="center">Modelo NLP para el análisis de sentimiento en review's de hoteles</h1>

#### [Enlace al conjunto de datos (kaggle)](https://www.kaggle.com/jiashenliu/515k-hotel-reviews-data-in-europe)

   ### Índice:
   
       1. Importación de librerías 
       
       2. Carga de los datos

       3. Creación de funciones
       
       4. Análisis preliminar
       
       5. Exploración y transformación de los datos
       
           5.1. Creación del campo 'Is_Positive_Review' 
           5.2. Selección de variables
           5.3. Longitud de palabras y de caracteres
           5.4. Distribución de longitud de las reviews según si son positivas o negativas
           5.5. Nube de palabras
           5.6. Normalización
           5.7. Análisis de sentimiento
               
               5.7.1. Con Vader
               5.7.2. Con TextBlob
               
       6. Vectorización del texto
       
       7. Exploración previa al modelado
       
       8. Construcción del modelo
       
           8.1. Balanceo de los datos
           8.2. Regresión logística
           8.3. Mejora del modelo
           
       9. Guardar modelo

## 1. Importación de librerías y datos

In [ ]:
# Instalacion previa de librerias empleadas
!pip install wordcloud # Generador de nube de palabras
!pip install textblob # Procesamiento de texto

In [ ]:
# Basicas
import pandas as pd # Analisis y manipulacion de datos
import numpy as np # Tratamiento de matrices
import matplotlib.pyplot as plt # Graficos
import seaborn as sns # Visualizacion de datos

### NLTK
import nltk # Procesamiento del lenguaje natural
nltk.download('averaged_perceptron_tagger') # Etiquetar las palabras
nltk.download('vader_lexicon') # Analisis de sentimiento
nltk.download('wordnet') # Categorizacion de las palabras
nltk.download('stopwords') # Quitar palabras comunes
from nltk.corpus import wordnet
from nltk import pos_tag # Clasificacion de palabras
from nltk.corpus import stopwords # Eliminar palabras vacias
from nltk.tokenize import WhitespaceTokenizer # Tokenizar
from nltk.stem import WordNetLemmatizer # Lematizar
from nltk.stem.wordnet import WordNetLemmatizer # Lematizar
from nltk.sentiment.vader import SentimentIntensityAnalyzer # Analisis de sentimiento

### TRATAMIENTO DE TEXTO
from wordcloud import WordCloud # Nube de palabras
import string # Operaciones de cadenas de caracteres
from textblob import TextBlob # Procesamiento del lenguaje

### SKLEARN
from sklearn.feature_extraction.text import TfidfVectorizer # Codificacion de documentos, segun frecuenca de las palabras
from sklearn.model_selection import train_test_split # Dividir los datos en entrenamiento y validacion
from imblearn.over_sampling import SMOTE # Balanceo de los datos
from sklearn.linear_model import LogisticRegression # Clasificador
from sklearn.ensemble import RandomForestClassifier # Clasificador
from sklearn.metrics import classification_report # Metricas para valoracion del modelo
from sklearn.metrics import f1_score, confusion_matrix # Metricas para valoracion del modelo
from sklearn.metrics import roc_curve, auc, roc_auc_score # Metricas para valoracion del modelo
from sklearn.metrics import plot_confusion_matrix # Metricas para valoracion del modelo
from sklearn.model_selection import GridSearchCV # Ajuste de hiper-parametros

import pickle # Guardar modelo

### ADICIONALES
import warnings # Control de advertencias
warnings.filterwarnings('ignore')
from tqdm import tqdm 
tqdm.pandas(desc='Processing Dataframe') # Barra de progreso

## 2. Carga de los datos

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Cargar dataset
df = pd.read_csv("/kaggle/input/515k-hotel-reviews-data-in-europe/Hotel_Reviews.csv")

## 3. Creación de las funciones que se utilizarán

In [ ]:
# Función para limpiar el texto
def limpiar_texto(texto):
    # Poner el texto en minúsculas
    texto = texto.lower()
    # Tokenizar el texto y quitar los signos de puntuación
    texto = [word.strip(string.punctuation) for word in texto.split(" ")]
    # Quitar las palabras que contengan números
    texto = [word for word in texto if not any(c.isdigit() for c in word)]
    # Quitar las stop words
    stop = stopwords.words('english')
    texto = [x for x in texto if x not in stop]
    # Quitar los tokens vacíos
    texto = [t for t in texto if len(t) > 0]
    # Pos tags
    pos_tags = pos_tag(texto)
    # Lematizar el texto
    texto = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # Quitar las palabras con sólo una letra
    texto = [t for t in texto if len(t) > 1]
    # Unir todo
    texto = " ".join(texto)
    return(texto)

# Función para dibujar la nube de palabras
def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color = 'white',
        max_words = 200,
        max_font_size = 40, 
        scale = 3,
        random_state = 42
    ).generate(str(data))

    fig = plt.figure(1, figsize = (20, 20))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize = 20)
        fig.subplots_adjust(top = 2.3)

    plt.imshow(wordcloud)
    plt.show()
    

# Etiquetado de nombres, verbos, adjetivos o adverbios
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
# Para el train_validate_test_split (para probar los modelos y mejorarlos)
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=101):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.loc[perm[:train_end]]
    validate = df.loc[perm[train_end:validate_end]]
    test = df.loc[perm[validate_end:]]
    return train, validate, test

## 4. Análisis preliminar

In [ ]:
print("El df tiene un total de {} columnas y un total de {} registros".format(df.shape[1], df.shape[0]))

In [ ]:
# Columnas df1
df.columns

**Analisis preliminar df**

El dataframe tiene la columna "Negative_Review", que recoge las reviews negativas y "Positive_Review", que recoge las reviews positivas. Por lo tanto, para recoger todas estas reviews habría que unirlas en un nuevo campo, el cual se llamará "Reviews".

Además, se creará el campo Is_Positive_Review donde si es 1, es que la review ha recibido un 5 o más, y, por lo tanto, es positiva, mientras que si es 0 es que la review ha recibido menos de un 5 de score y por lo tanto es negativa. Esto es porque en este dataframe las puntuaciones van del 0 al 10.

## 5. Exploración y transformación de los datos

#### 5.1 Creación del campo 'Is_Positive_Review' 

Se crea el campo Is_Positive_Review (si es positiva = 1, si es negativa = 0)

In [ ]:
df

In [ ]:
# En Reviews se añaden los dos tipos de Review concatenadas con un espacio
df["Reviews"] = df["Positive_Review"] + " " + df["Negative_Review"]

# Se quitan los "No Negative" y "No Positive" de las reviews
df["Reviews"] = df["Reviews"].astype(str)
df["Reviews"] = df["Reviews"].apply(lambda x: x.replace("No Negative", "").replace("No Positive", ""))

# Se crea la columna "Is_Positive_Review" para aquellas Review cuyo score es mayor a 5. 
df['Is_Positive_Review'] = df['Reviewer_Score'].progress_apply(lambda x: 1 if x >= 5 else 0)

In [ ]:
print("El df tiene un conjunto de {} opiniones positivas y un conjunto de {} opiniones negativas".format(df["Is_Positive_Review"].value_counts()[1], df["Is_Positive_Review"].value_counts()[0]))
print("En porcentaje, el {0:.2f}% de las reviews son positivas y el {1:.2f}% de las reviews son negativas".format(df["Is_Positive_Review"].value_counts(normalize=True)[1]*100, df["Is_Positive_Review"].value_counts(normalize=True)[0]*100))

Como se puede observar, el dataset está claramente desbalanceado, cuestión que será importante y que hay que tener en cuenta al entrenar el modelo.

#### 5.2 Selección de variables

Las variables que se van a seleccionar para el dataframe van a ser las siguientes: Reviews, Is_Positive_Review.

In [ ]:
# Selección de las variables de interés
df = df.loc[:, ['Reviews', 'Is_Positive_Review']]

print("Dimensiones de df1: {}".format(df.shape))

#### 5.3 Longitud de palabras y de caracteres

In [ ]:
# Añadir número de caracteres
df["Caracteres_len"] = df["Reviews"].progress_apply(lambda x: len(x))

# Añadir número de palabras
df["Palabras_len"] = df["Reviews"].progress_apply(lambda x: len(x.split(" ")))

#### 5.4 Distribución de longitud de las reviews según si son positivas o negativas

In [ ]:
# Para el número de caracteres
fig = plt.figure(figsize=(10,6))
plt1 = sns.distplot(df[df["Is_Positive_Review"]==0].Caracteres_len, hist=True)
plt2 = sns.distplot(df[df["Is_Positive_Review"]==1].Caracteres_len, hist=True)
fig.legend(labels=['Review negativa','Review positiva'])
plt.show()

In [ ]:
# Para el número de palabras
fig = plt.figure(figsize=(10,6))
plt1 = sns.distplot(df[df["Is_Positive_Review"]==0].Palabras_len, hist=True)
plt2 = sns.distplot(df[df["Is_Positive_Review"]==1].Palabras_len, hist=True)
fig.legend(labels=['Review negativa','Review positiva'])
plt.show()

Como se puede observar no hay mucha diferencia en cuanto a longitud (tanto de caracteres como de palabras) en las reviews negativas y positivas. Por el motivo citado, no se incorporarán estas variable en el modelo, puesto que no parecen ayudar a ditinguir entre valoraciones positivas y negativas.

#### 5.5 Nube de palabras

In [ ]:
# Se imprime la nube de palabras con la función cargada anteriormente
show_wordcloud(df["Reviews"])

#### 5.6 Normalización

Para este apartado se aplicará la función definida previamente de limpieza de texto. Aplicando esa función al conjunto de datos, lo que se hará será:

- Poner el texto en minúscula
- Tokenizar el texto
- Quitar las palabras que contengan números
- Quitar las stop words
- Quitar los tokens vacíos
- Lematizar el texto
- Quitar las palabras con una letra

In [ ]:
# Se aplica la función anterior
df['Reviews_procesadas'] = df['Reviews'].progress_apply(lambda x: limpiar_texto(x))

#### 5.7 Análisis de sentimiento

El análisis de sentimiento, para que sea más completo, se va a realizar con dos librerías diferentes. La primera va a ser con Vader, del módulo nltk. Vader proporciona cuatro nuevos campos (score positivo, score negativo, score neutro y un score llamado compound que integra todos los score anteriores, esto es, que cuanto más positivo, más positiva es la review mientras que cuanto más negativo, más negativa será la review). La segunda librería será la de TextBlob. TextBlob proporciona una tupla con dos: polaridad y subjetividad. La polaridad va del valor -1 hasta el 1. Si es negativa significa que tiene sentimientos negativos mientras que si es positiva significa que tiene sentimientos positivos. La subjetividad va entre 0 y 1, y cuantifica la opinión personal contenida en el texto. Una mayor subjetividad significa que el texto contiene mucha más opinión personal que una información objetiva.

Para extraer el sentimiento se utilizarán las reviews sin procesar.

#### 5.7.1 Con Vader

In [ ]:
# Con Vader, del módulo nltk. Este módulo añade un score positivo, negativo, neutro y una integración de todas las anteriores
analizador = SentimentIntensityAnalyzer()
df["Sentimiento"] = df["Reviews"].progress_apply(lambda x: analizador.polarity_scores(x))
df = pd.concat([df.drop(['Sentimiento'], axis=1), df['Sentimiento'].apply(pd.Series)], axis=1)

#### 5.7.2 Con TextBlob

In [ ]:
# Con TextBlob
df['Polaridad'] = df['Reviews'].progress_apply(lambda x: TextBlob(x).sentiment.polarity) 
df['Subjetividad'] = df['Reviews'].progress_apply(lambda x: TextBlob(x).sentiment.subjectivity) 

## 6. Vectorización del texto

Se van a extraer las características del texto utilizando TFIDFVectorizer. En este caso se quiere:

- Utilizar como máximo 200 características
- Unigramas, bigramas, trigramas y cuatrigramas
- Que el sistema ignore los elemenos que al menos no aparezcan en 3 reviews
- Que, puesto que el texto ya está tokenizado, no utilice la función tokenizadora de Scikit-Learn

In [ ]:
vectorizador = TfidfVectorizer(ngram_range = (1,4), min_df = 3, max_features = 200,
                               use_idf = True, smooth_idf = True, norm = 'l2') 

vector_corpus = vectorizador.fit_transform(df['Reviews_procesadas'].to_list()) 

type(vector_corpus) # Ya está transformado el texto.

## 7. Exploración previa al modelado

Como se ha comentado previamente, al realizar el primer análisis exploratorio, el texto está desbalanceado. Sin embargo, además de esa conclusión se pueden sacar otras conclusiones mediante la exploración de los resultados que tenemos actualmente antes de construir cualquier modelo.

In [ ]:
# Reviews con positividad más alta en cuanto a sentimientos según Vader (más de 10 palabras)
df[df["Palabras_len"] >= 10].sort_values("pos", ascending = False)[["Reviews", "pos"]].head(10)

In [ ]:
# Reviews con negatividad más alta en cuanto a sentimientos según Vader (más de 10 palabras)
df[df["Palabras_len"] >= 10].sort_values("neg", ascending = False)[["Reviews", "neg"]].head(10)

Se puede observar que Vader interpreta 'no' y 'nothing' como negativo, mientras que muchas veces no significa algo negativo. Por ejemplo si se comenta que no se ha tenido ningún problema con el hotel. Sin embargo, afortunadamente, la gran mayoría de reviews son negativas de verdad.

## 8. Construcción del modelo

In [ ]:
# Selección de variables para la construcción del modelo (en resumen, sólo nos quedamos con los campos numéricos)
target = 'Is_Positive_Review'
campos_ignorar = ['Reviews', 'Reviews_procesadas', 'Caracteres_len', 'Palabras_len']
campos_features = [i for i in df.columns if i not in campos_ignorar]

# División en conjuntos de test, validate y train
train, validate, test = train_validate_test_split(df[campos_features])
X_train = train.drop('Is_Positive_Review',1)
y_train = train['Is_Positive_Review']
X_validation = validate.drop('Is_Positive_Review', 1)
y_validation = validate['Is_Positive_Review']
X_test = test.drop('Is_Positive_Review', 1)
y_test = test['Is_Positive_Review']

In [ ]:
campos_features

#### 8.1 Balanceo de los datos

Ya se ha visto que están desbalanceados, con muchas más reviews positivas que negativas. Los datos se balancearán con la librería SMOTE. 

In [ ]:
# SMOTE para balancear los datos (se aplica sólo en el conjunto de entrenamiento)
balancear = SMOTE(random_state = 41)
X_train_balanceado, y_train_balanceado = balancear.fit_resample(X_train, y_train) # Se aplica al conjunto de train

# Se crean los dataframes con los conjuntos balanceados
X_train_balanceado = pd.DataFrame(data = X_train_balanceado, columns = X_train.columns) # X_train_balanceado
Y_train_balanceado = pd.DataFrame(data= y_train_balanceado, columns = ['Is_Positive_Review'])

#### 8.2 Regresión logística

In [ ]:
# Regresión logística
reg_logistica = LogisticRegression()
reg_logistica.fit(X_train_balanceado, Y_train_balanceado)

In [ ]:
# Predicción
y_pred = reg_logistica.predict(X_test)

print('Precisión de la regresión logística en el test: {:.2f}'.format(reg_logistica.score(X_test, y_test)))

In [ ]:
# Matriz de confusión
plot_confusion_matrix(reg_logistica, X_test, y_test, normalize = None)

print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred)) # Por si no se ve bien en el dibujo

In [ ]:
# ROC
y_pred = [x[1] for x in reg_logistica.predict_proba(X_test)]
fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label = 1)

roc_auc = auc(fpr, tpr)

plt.figure(1, figsize = (15, 10))
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

#### 8.3 Mejora del modelo

In [ ]:
# Mejorar las características de la regresión con otros parámetros. 
# En este caso se sacarán los mejores valores para, sobre todo, threshold. En regresión logística el valor predeterminado es
# de 0.5

fpr, tpr, thresholds = roc_curve(y_validation, reg_logistica.predict_proba(X_validation)[:,1])
i = np.arange(len(tpr)) 
roc = pd.DataFrame({'fpr' : pd.Series(fpr, index=i),
                    'tpr' : pd.Series(tpr, index = i), 
                    '1-fpr' : pd.Series(1-fpr, index = i), 
                    'tf' : pd.Series(tpr - (1-fpr), index = i), 
                    'thresholds' : pd.Series(thresholds, index = i)})

roc.iloc[(roc.tf-0).abs().argsort()[:1]]

En las regresiones logísticas, el parámetro threshold viene predeterminado con un valor de 0.5. En este caso, según lo anterior, el óptimo es 0.482. Vamos a probar con este parámetro, a ver si mejora la precisión.

In [ ]:
# Threshold = 0.467
threshold = 0.467
preds = np.where(reg_logistica.predict_proba(X_test)[:,1] > threshold, 1, 0)
print('Precisión: {:.2f}'.format(reg_logistica.score(X_test, preds)))

Se ha mejorado muchísimo la precisión (ahora es 0.99). Se va a intentar mejorar más aún el modelo con el tuneo de los hiperparámetros (GridSearch)

In [ ]:
# GridSearch

grid = {"C":np.array([0.001,0.01,0.1,1,10]), "penalty":["l1","l2"]}

reg_logistica_gridsearchcv = GridSearchCV(reg_logistica, grid, cv=10)

reg_logistica_gridsearchcv.fit(X_validation, y_validation)
print('Mejor Penalty:', reg_logistica_gridsearchcv.best_estimator_.get_params()['penalty'])
print('Mejor C:', reg_logistica_gridsearchcv.best_estimator_.get_params()['C'])

In [ ]:
# Aplicación del GridSearch y del Threshold óptimo

regr_log = LogisticRegression(penalty='l2', C=0.01) # Se ponen el mejor penalty y el mejor C calculados previamente

regr_log.fit(X_train_balanceado, y_train_balanceado) # Entrenamos el modelo con los datos balanceados

threshold = 0.482 # óptimo, ya guardado previamente
prediccion = np.where(regr_log.predict_proba(X_test)[:,1] > threshold, 1, 0) # Predicción con los valores óptimos

print(classification_report(y_test, prediccion))
print('Precisión de la regresión logística en el test: {:.2f}'.format(regr_log.score(X_test, prediccion)))

In [ ]:
# Matriz de confusión
plot_confusion_matrix(regr_log, X_test, y_test, normalize = None)

print(classification_report(y_test, prediccion))
print(confusion_matrix(y_test, prediccion))

In [ ]:
# ROC
y_pred = [x[1] for x in regr_log.predict_proba(X_test)]
fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label = 1)

roc_auc = auc(fpr, tpr)

plt.figure(1, figsize = (15, 10))
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

## 9. Guardar modelo

In [ ]:
# Guardar el modelo (descomentar para guardarlo en formato .pkl)
"""
with open("./model_sentiment_analysis.pkl", 'wb') as f:
    pickle.dump(regr_log, f)
"""